In [1]:
import sys
sys.path.insert(0,'..')
from src.server.db_api import connect

In [2]:
import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

[Ani] client initialize EVAConnection


In [3]:
cursor = connection.cursor()

[Ani] client initialize EVACursor


In [4]:
cursor.execute('UPLOAD INFILE "../data/ua_detrac/ua_detrac.mp4" PATH "video.mp4";')
response = cursor.fetch_all()
print(response)

[Ani] client in function execute_async
[Ani] in function send_message. Sending message to server
Response Object:
@status: 0
@batch: Batch Object:
@dataframe: Empty DataFrame
Columns: []
Index: []
@batch_size: 0
@identifier_column: id
@metrics: None


In [ ]:
from ipywidgets import Video
Video.from_file("../data/ua_detrac/ua_detrac.mp4", embed=True)

In [6]:
cursor.execute('LOAD DATA INFILE "video.mp4" INTO MyVideo;')
response = cursor.fetch_all()
print(response)

[Ani] client in function execute_async
[Ani] in function send_message. Sending message to server
Response Object:
@status: 0
@batch: Batch Object:
@dataframe:        Video  Num Loaded Frames
0  video.mp4                252
@batch_size: 1
@identifier_column: id
@metrics: None


In [7]:
cursor.execute("""CREATE UDF IF NOT EXISTS MidasDepthEstimator
      INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
      OUTPUT (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
      TYPE  Classification
      IMPL  'src/udfs/midas_depth_estimator.py';
      """)
response = cursor.fetch_all()
print(response)

[Ani] client in function execute_async
[Ani] in function send_message. Sending message to server
Response Object:
@status: 0
@batch: Batch Object:
@dataframe: Empty DataFrame
Columns: []
Index: []
@batch_size: 0
@identifier_column: id
@metrics: None


In [8]:
cursor.execute("""SELECT id, MidasDepthEstimator(data) FROM MyVideo""")
response = cursor.fetch_all()

[Ani] client in function execute_async
[Ani] in function send_message. Sending message to server


In [9]:
response.batch.frames

,id,frames
0,210,"[[79.1489181519, 76.156829834, 78.8829116821, ..."
1,211,"[[81.7383804321, 78.4569473267, 81.3514633179,..."
2,212,"[[84.4806900024, 81.265914917, 83.527961731, 8..."
3,213,"[[89.7282333374, 86.7491378784, 88.6310501099,..."
4,214,"[[91.0610961914, 88.1939697266, 89.8064117432,..."
...,...,...
247,95,"[[55.4163398743, 53.1123046875, 54.0328292847,..."
248,96,"[[62.0665588379, 59.7210502625, 60.9966545105,..."
249,97,"[[69.4775085449, 67.0294570923, 68.5536651611,..."
250,98,"[[63.2394065857, 60.547000885, 61.762676239, 6..."


In [10]:
output_width = len(response.batch.frames.iloc[0][1])
output_height = len(response.batch.frames.iloc[0][1][0])

print(f'output dimension: {output_width}x{output_height}')

output dimension: 90x160
